### 필요 패키지 설치

In [1]:
!pip install wandb

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 8.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 751.2/751.2 kB 19.9 MB/s eta 0:00:00


### wandb 로그인


wandb에 로그인하여 api키를 입력합니다.

In [2]:
import wandb

# WanDB 로그인
wandb.login()

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /data/ephemeral/home/.netrc


True

### hyperparameter_tune

튜닝할 파라미터의 이름, 자료형, 범위 등 세팅합니다.

In [ ]:
# 파라미터 세팅 구현

sweep_config = {
    'method': 'random',
    'metric': {
        'name': 'accuracy',
        'goal': 'maximize'
    },
    'parameters': {
        'lg_max_iter' : {
            'values' : [300,500,700]
        },
        'lg_class_weight' : {
            'values' : [None,'balanced']
        },
        'rf_n_estimators': {
            'values': [50, 100, 200]
        },
        'rf_max_depth': {
            'values': [10, 20, 30]
        },
        'rf_min_samples_split': {
            'values': [2, 5, 10]
        },
        'dc_min_samples_split': {
            'values': [2, 4, 8]
        },
        'dc_max_depth': {
            'values': [10, 20, 40]
        },
        'knn_n_neighbors': {
            'values': [4, 6, 8]
        },
        'knn_leaf_size': {
            'values': [20, 30, 40]
        }
    }
}

sweep_id = wandb.sweep(
    sweep_config,
    project="hyperparameter_tune"
)
sweep_id

#### sweep 학습

In [ ]:
####임시로 학습 데이터 불러오는 블럭


In [ ]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, f1_score
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier

import numpy as np


def soft_voting(predictions):
    voting_result = np.argmax(np.sum(predictions, axis=0), axis=1)
    return voting_result


def wandb_training_function():
    with wandb.init() as run:
        params = wandb.config

        val_scores = list()

        lg = LogisticRegression(max_iter=params['lg_max_iter'],
                                class_weight=params['lg_class_weight'])

        # 모델 정의
        rf = RandomForestClassifier(n_estimators=params['rf_n_estimators'],
                                max_depth=params['rf_max_depth'],
                                min_samples_split=params['rf_min_samples_split'])

        dc = DecisionTreeClassifier(min_samples_split=params['dc_min_samples_split'],
                                    max_depth=params['dc_max_depth'])

        knn = KNeighborsClassifier(n_neighbors=params['knn_n_neighbors'],
                                    leaf_size=params['knn_leaf_size'])

        # 모델 리스트 구성
        models = [lg ,rf, dc, knn]

        # 모델 학습
        for model in models:
            model.fit(x_train, y_train)

        # 모델 예측
        preds_train = [model.predict_proba(x_train) for model in models]
        preds_valid = [model.predict_proba(x_valid) for model in models]
        voting_preds_train = soft_voting(preds_train)
        voting_preds_valid = soft_voting(preds_valid)

        # 모델 평가
        train_accuracy = accuracy_score(y_train, voting_preds_train)
        valid_accuracy = accuracy_score(y_valid, voting_preds_valid)

        val_scores.append(valid_accuracy)

        wandb.log({"train_accuracy": train_accuracy, "valid_accuracy": valid_accuracy})

        metrics = {"cv_metric": np.mean(val_scores)}
        wandb.log(metrics)

In [ ]:
count = 20

wandb.agent(sweep_id, function=wandb_training_function, count=count)

### 하이퍼 파라미터 분석

ensemble에 사용할 모델인 KNeighborsClassifier, DecisionTreeClassifier, RandomForestClassifier, LogisticRegression에 대한 hyperparameter 최적화를 진행하였음.
아래는 각 모델별 학습 그래프이다.

<center>
<img src='hyperparameter_img/dc.png'/><br>
<img src='hyperparameter_img/knn.png'/><br>
<img src='hyperparameter_img/lg.png'/><br>
<img src='hyperparameter_img/rf.png'/><br>
<img src='hyperparameter_img/all.png'/><br>

</center>

아래는 0.4224의 정확도를 가진 파라미터 값이다.

KNeighborsClassifier(leat_size = 20, n_neighbors = 4)
DecisionTreeClassifier(max_depth = 20, min_samples_split=2)
RandomForestClassifier(max_depth = 10,min_samples_split = 10, n_estimators = 50)
LogisticRegression(class_weight=None, max_iter=300)

위 파라미터로 soft-vote-ensamble을 진행하였으나 이전보다 더 낮은 acc를 도출하였다.
